In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import transforms, datasets

import numpy as np
import matplotlib.pyplot as plt

In [4]:
train = datasets.FashionMNIST('data/', train=True, download=True,
                             transform=transforms.Compose([transforms.ToTensor()]))

test = datasets.FashionMNIST('data/', train=True, download=True,
                            transform=transforms.Compose([transforms.ToTensor()]))

# Define the Network class

In [52]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
        #layer 1 : input layer
        t = t
        #Layer 2 : Conv layer 1
        t = self.conv1(t)
        t = nn.functional.relu(t)
        t = nn.functional.max_pool2d(t,kernel_size=2, stride=2)
        #layer 3 : Conv layer 2
        t = self.conv2(t)
        t = nn.functional.relu(t)
        t = nn.functional.max_pool2d(t, kernel_size=2, stride=2)
        #layer 4 : Linear layer 1
        t = t.reshape(-1,12*4*4)
        t = self.fc1(t)
        t = nn.functional.relu(t)
        #layer 5 : Linear layer 2
        t = self.fc2(t)
        t = nn.functional.relu(t)
        #layer 6 : Output layer 
        t = self.out(t)
        #t = nn.functional.softmax(t, dim=1)
        
        return t
        

In [53]:
network = NeuralNet()
for name,param in network.named_parameters():
    print(name,'\t\t',param.shape)

conv1.weight 		 torch.Size([6, 1, 5, 5])
conv1.bias 		 torch.Size([6])
conv2.weight 		 torch.Size([12, 6, 5, 5])
conv2.bias 		 torch.Size([12])
fc1.weight 		 torch.Size([120, 192])
fc1.bias 		 torch.Size([120])
fc2.weight 		 torch.Size([60, 120])
fc2.bias 		 torch.Size([60])
out.weight 		 torch.Size([10, 60])
out.bias 		 torch.Size([10])


In [30]:
train_set = torch.utils.data.DataLoader(train,batch_size=100)

In [31]:
batch = next(iter(train_set))

In [55]:
images, labels = batch

# Creating Loss

In [56]:
predictions = network(images)
loss = nn.functional.cross_entropy(predictions, labels)
loss.item()

2.2988078594207764

# Calculate gradient


In [59]:
print(network.conv1.weight.grad)

None


In [60]:
loss.backward() #Calculate gradient

In [62]:
print(network.conv1.weight.grad.shape)

torch.Size([6, 1, 5, 5])


# Update weights

In [63]:
optimizer = optim.Adam(network.parameters(),lr=0.01)

In [64]:
loss.item()

2.2988078594207764

In [68]:
predictions.argmax(dim=1).eq(labels).sum().item()

12

In [69]:
optimizer.step() #updates the weights

In [73]:
predictions = network(images)
loss = nn.functional.cross_entropy(predictions,labels)
loss.item()

2.272934913635254